# Healthcare Insurance Risk Model - Executive Validation Report

**Purpose**: Comprehensive validation report combining technical, business, and clinical validation for stakeholder review.

**Audience**: Executives, Product Managers, Compliance Officers, Clinical Stakeholders

**Report Sections**:
1. Executive Summary
2. Technical Validation
3. Business Performance
4. Clinical Relevance
5. Compliance & Governance
6. Recommendations & Actions

In [ ]:
# Setup and configuration
import json
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import mlflow
from mlflow.tracking import MlflowClient

# Configure parameters
dbutils.widgets.text("catalog", "juan_dev", "Unity Catalog")
dbutils.widgets.text("ml_schema", "healthcare_data", "ML Schema")
dbutils.widgets.text("model_name", "insurance_model", "Model Name")
dbutils.widgets.text("report_period_days", "30", "Report Period (days)")

catalog = dbutils.widgets.get("catalog")
ml_schema = dbutils.widgets.get("ml_schema")
model_name = dbutils.widgets.get("model_name")
report_period = int(dbutils.widgets.get("report_period_days"))

# Configure MLflow
mlflow.set_registry_uri("databricks-uc")
client = MlflowClient()

# Table references
predictions_table = f"{catalog}.{ml_schema}.ml_patient_predictions"
full_model_name = f"{catalog}.{ml_schema}.{model_name}"

print(f"{'='*80}")
print(f"HEALTHCARE INSURANCE RISK MODEL - EXECUTIVE VALIDATION REPORT")
print(f"{'='*80}")
print(f"Model: {full_model_name}")
print(f"Report Period: Last {report_period} days")
print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"{'='*80}\n")

## 1. Executive Summary

High-level overview of model performance, business impact, and key findings.

In [ ]:
print("\n" + "="*80)
print("EXECUTIVE SUMMARY")
print("="*80)

# Load predictions data
predictions_df = spark.table(predictions_table)
cutoff_date = (datetime.now() - timedelta(days=report_period)).strftime('%Y-%m-%d')
recent_predictions = predictions_df.filter(col("prediction_timestamp") >= cutoff_date)

# Key metrics for executive summary
exec_summary = recent_predictions.agg(
    countDistinct("patient_natural_key").alias("unique_patients"),
    count("*").alias("total_predictions"),
    avg("adjusted_prediction").alias("avg_risk_score"),
    sum(when(col("high_risk_patient") == True, 1).otherwise(0)).alias("high_risk_count")
).collect()[0]

high_risk_pct = (exec_summary.high_risk_count / exec_summary.total_predictions) * 100

# Business impact estimates
avg_intervention_cost = 500
avg_prevented_claim = 5000
success_rate = 0.30

intervention_costs = exec_summary.high_risk_count * avg_intervention_cost
prevented_value = exec_summary.high_risk_count * success_rate * avg_prevented_claim
net_benefit = prevented_value - intervention_costs
roi = (net_benefit / intervention_costs * 100) if intervention_costs > 0 else 0

print(f"\n📊 Key Performance Indicators:")
print(f"  • Patients Evaluated: {exec_summary.unique_patients:,}")
print(f"  • Total Risk Assessments: {exec_summary.total_predictions:,}")
print(f"  • Average Risk Score: {exec_summary.avg_risk_score:.1f}/100")
print(f"  • High-Risk Patients Identified: {exec_summary.high_risk_count:,} ({high_risk_pct:.1f}%)")

print(f"\n💰 Estimated Business Impact:")
print(f"  • Intervention Costs: ${intervention_costs:,.0f}")
print(f"  • Prevented Claims Value: ${prevented_value:,.0f}")
print(f"  • Net Benefit: ${net_benefit:,.0f}")
print(f"  • Return on Investment: {roi:.0f}%")

# Model status
try:
    champion_info = client.get_model_version_by_alias(full_model_name, "champion")
    run_data = client.get_run(champion_info.run_id)
    r2_score = run_data.data.metrics.get("r2_score", 0)
    mae = run_data.data.metrics.get("mean_absolute_error", 0)
    
    print(f"\n🤖 Model Performance:")
    print(f"  • Champion Version: {champion_info.version}")
    print(f"  • Model Accuracy (R²): {r2_score:.3f}")
    print(f"  • Prediction Error (MAE): {mae:.2f}")
    print(f"  • Status: {champion_info.status}")
except Exception as e:
    print(f"\n⚠️  Model Status: Unable to retrieve champion model information")

print(f"\n✅ Overall Assessment:")
if roi > 100 and high_risk_pct >= 5 and high_risk_pct <= 25:
    print(f"  Model is performing EXCELLENTLY and delivering strong business value.")
    overall_status = "EXCELLENT"
elif roi > 50:
    print(f"  Model is performing WELL and delivering positive business value.")
    overall_status = "GOOD"
elif roi > 0:
    print(f"  Model is ACCEPTABLE but has room for improvement.")
    overall_status = "ACCEPTABLE"
else:
    print(f"  Model requires ATTENTION and optimization.")
    overall_status = "NEEDS_IMPROVEMENT"

print("="*80)

## 2. Technical Validation

Data quality, prediction accuracy, and technical health checks.

In [ ]:
print("\n" + "="*80)
print("TECHNICAL VALIDATION")
print("="*80)

# Run data validation
data_validation_result = dbutils.notebook.run(
    "./data_validation",
    timeout_seconds=300,
    arguments={
        "catalog": catalog,
        "schema": ml_schema.replace("healthcare_data", "healthcare_data"),
        "validation_date": datetime.now().strftime('%Y-%m-%d')
    }
)

print(f"\n✅ Data Validation Results:")
try:
    validation_data = json.loads(data_validation_result)
    print(f"  Status: {validation_data.get('status', 'UNKNOWN')}")
    if validation_data.get('status') == 'SUCCESS':
        print(f"  ✓ All prerequisite tables validated")
        print(f"  ✓ Required columns present")
        print(f"  ✓ Data quality checks passed")
    else:
        print(f"  ⚠️  {validation_data.get('message', 'Validation issues detected')}")
except Exception as e:
    print(f"  ℹ️  Validation completed with status: {data_validation_result}")

# Data quality metrics - check if confidence interval columns exist
has_confidence_cols = "confidence_interval_upper" in recent_predictions.columns

if has_confidence_cols:
    quality_metrics = recent_predictions.agg(
        (sum(when(col("adjusted_prediction").isNull(), 1).otherwise(0)) / count("*") * 100).alias("null_pct"),
        (sum(when(col("adjusted_prediction") < 0, 1).otherwise(0)) / count("*") * 100).alias("negative_pct"),
        (sum(when(col("adjusted_prediction") > 100, 1).otherwise(0)) / count("*") * 100).alias("outlier_pct"),
        avg(col("confidence_interval_upper") - col("confidence_interval_lower")).alias("avg_ci")
    ).collect()[0]
else:
    quality_metrics = recent_predictions.agg(
        (sum(when(col("adjusted_prediction").isNull(), 1).otherwise(0)) / count("*") * 100).alias("null_pct"),
        (sum(when(col("adjusted_prediction") < 0, 1).otherwise(0)) / count("*") * 100).alias("negative_pct"),
        (sum(when(col("adjusted_prediction") > 100, 1).otherwise(0)) / count("*") * 100).alias("outlier_pct")
    ).collect()[0]

print(f"\n📊 Prediction Quality Metrics:")
print(f"  • Data Completeness: {100 - quality_metrics.null_pct:.2f}%")
print(f"  • Outliers: {quality_metrics.outlier_pct:.2f}%")
if has_confidence_cols:
    print(f"  • Avg Confidence Interval: ±{quality_metrics.avg_ci:.2f}")

if quality_metrics.null_pct < 1 and quality_metrics.outlier_pct < 1:
    print(f"  ✅ Excellent data quality")
elif quality_metrics.null_pct < 5:
    print(f"  ✓ Acceptable data quality")
else:
    print(f"  ⚠️  Data quality requires attention")

# Prediction volume trends
print(f"\n📈 Prediction Volume Trends:")
daily_volume = recent_predictions.groupBy(
    date_format(col("prediction_timestamp"), "yyyy-MM-dd").alias("date")
).agg(
    count("*").alias("prediction_count")
).orderBy("date", ascending=False).limit(7).collect()

print(f"  Last 7 days:")
for row in daily_volume:
    print(f"    {row.date}: {row.prediction_count:,} predictions")

# Use builtins.sum to avoid conflict with PySpark's sum function
import builtins
avg_daily = builtins.sum([row.prediction_count for row in daily_volume]) / len(daily_volume) if daily_volume else 0
print(f"  • Average daily volume: {avg_daily:,.0f} predictions")

print("="*80)

## 3. Business Performance

Risk distribution, high-risk identification, and business KPIs.

In [ ]:
print("\n" + "="*80)
print("BUSINESS PERFORMANCE")
print("="*80)

# Risk category distribution
print(f"\n📊 Risk Category Distribution:")
risk_dist = recent_predictions.groupBy("risk_category").agg(
    count("*").alias("count")
).orderBy("risk_category").collect()

# Use builtins.sum to avoid conflict with PySpark's sum
import builtins
total = builtins.sum([row['count'] for row in risk_dist])
for row in risk_dist:
    pct = (row['count'] / total) * 100
    bar = "█" * int(pct / 2)  # Visual bar chart
    print(f"  {row.risk_category:>10}: {row['count']:>7,} ({pct:>5.1f}%) {bar}")

# Business KPI validation
print(f"\n✅ Business KPI Validation:")
low_pct = next((row['count'] / total * 100 for row in risk_dist if row.risk_category == 'low'), 0)
critical_pct = next((row['count'] / total * 100 for row in risk_dist if row.risk_category == 'critical'), 0)

print(f"  • Low-risk patients: {low_pct:.1f}% (Target: >30%)")
if low_pct >= 30:
    print(f"    ✓ Healthy baseline population")
else:
    print(f"    ⚠️  Lower than expected - investigate population health")

print(f"  • High-risk patients: {high_risk_pct:.1f}% (Target: 5-25%)")
if 5 <= high_risk_pct <= 25:
    print(f"    ✓ Appropriate high-risk identification rate")
elif high_risk_pct < 5:
    print(f"    ⚠️  May be under-identifying at-risk patients")
else:
    print(f"    ⚠️  May be over-flagging patients (alert fatigue risk)")

print(f"  • Critical cases: {critical_pct:.1f}% (Target: <10%)")
if critical_pct < 10:
    print(f"    ✓ Critical case rate within acceptable range")
else:
    print(f"    ⚠️  High critical rate - verify population or model calibration")

# Intervention candidates
print(f"\n🎯 Intervention Opportunities:")
high_risk_df = recent_predictions.filter(col("high_risk_patient") == True)
intervention_breakdown = high_risk_df.groupBy("risk_category").agg(
    count("*").alias("count"),
    avg("adjusted_prediction").alias("avg_risk")
).orderBy(col("avg_risk").desc()).collect()

print(f"  Total intervention candidates: {exec_summary.high_risk_count:,}")
for row in intervention_breakdown:
    print(f"    • {row.risk_category}: {row['count']:,} patients (avg risk: {row.avg_risk:.1f})")

print("="*80)

## 4. Clinical Relevance

Validate that predictions align with medical knowledge and healthcare best practices.

In [ ]:
print("\n" + "="*80)
print("CLINICAL RELEVANCE VALIDATION")
print("="*80)

# Smoking impact
print(f"\n🚬 Smoking Status Impact:")
smoking_analysis = recent_predictions.groupBy("patient_smoking_status").agg(
    count("*").alias("count"),
    avg("adjusted_prediction").alias("avg_risk"),
    (sum(when(col("high_risk_patient") == True, 1).otherwise(0)) / count("*") * 100).alias("high_risk_pct")
).collect()

for row in smoking_analysis:
    print(f"  {row.patient_smoking_status:>5}: Avg Risk={row.avg_risk:>5.1f}, High-Risk={row.high_risk_pct:>5.1f}%, N={row['count']:>6,}")

smoker_risks = {row.patient_smoking_status: row.avg_risk for row in smoking_analysis}
if "yes" in smoker_risks and "no" in smoker_risks:
    if smoker_risks["yes"] > smoker_risks["no"]:
        impact = smoker_risks["yes"] - smoker_risks["no"]
        print(f"  ✅ Smokers have {impact:.1f} points higher risk (clinically valid)")
    else:
        print(f"  ❌ Smoking NOT properly weighted - model calibration issue")

# Age correlation
print(f"\n👴 Age Category Analysis:")
age_analysis = recent_predictions.groupBy("patient_age_category").agg(
    count("*").alias("count"),
    avg("adjusted_prediction").alias("avg_risk")
).orderBy("patient_age_category").collect()

prev_risk = 0
age_valid = True
for row in age_analysis:
    trend = "↗" if row.avg_risk > prev_risk else "↘" if row.avg_risk < prev_risk else "→"
    print(f"  {row.patient_age_category:>15}: {row.avg_risk:>5.1f} {trend} (N={row['count']:,})")
    if row.avg_risk < prev_risk:
        age_valid = False
    prev_risk = row.avg_risk

if age_valid:
    print(f"  ✅ Age positively correlated with risk (clinically valid)")
else:
    print(f"  ⚠️  Age correlation inconsistent - review model features")

# BMI impact - check if column exists
has_bmi_category = "bmi_category" in recent_predictions.columns
if has_bmi_category:
    print(f"\n⚖️  BMI Category Impact:")
    bmi_analysis = recent_predictions.groupBy("bmi_category").agg(
        count("*").alias("count"),
        avg("adjusted_prediction").alias("avg_risk")
    ).orderBy("bmi_category").collect()

    for row in bmi_analysis:
        print(f"  {row.bmi_category:>12}: Avg Risk={row.avg_risk:>5.1f} (N={row['count']:,})")

    bmi_risks = {row.bmi_category: row.avg_risk for row in bmi_analysis}
    if "obese" in bmi_risks and "normal" in bmi_risks:
        if bmi_risks["obese"] > bmi_risks["normal"]:
            print(f"  ✅ Obesity correctly associated with higher risk")
        else:
            print(f"  ⚠️  BMI relationship unclear - verify feature engineering")
else:
    bmi_risks = {}
    print(f"\n⚖️  BMI Category: Column not available in predictions table")

print(f"\n📋 Clinical Validation Summary:")
clinical_checks = [
    ("Smoking impact", smoker_risks.get("yes", 0) > smoker_risks.get("no", 0) if "yes" in smoker_risks and "no" in smoker_risks else False),
    ("Age correlation", age_valid)
]
if has_bmi_category:
    clinical_checks.append(("BMI impact", bmi_risks.get("obese", 0) > bmi_risks.get("normal", 0) if "obese" in bmi_risks and "normal" in bmi_risks else False))

import builtins
passed_clinical = builtins.sum([1 for _, result in clinical_checks if result])
for check_name, result in clinical_checks:
    print(f"  {'✓' if result else '✗'} {check_name}")

print(f"\n  Clinical validation score: {passed_clinical}/{len(clinical_checks)} checks passed")

print("="*80)

## 5. Regional Equity & Fairness

Ensure predictions are fair and equitable across demographics and regions.

In [ ]:
print("\n" + "="*80)
print("REGIONAL EQUITY & FAIRNESS ANALYSIS")
print("="*80)

# Import builtins to avoid conflicts with PySpark functions
import builtins

# Regional analysis - check if region column exists
has_region = "patient_region" in recent_predictions.columns or "region" in recent_predictions.columns
region_col = "patient_region" if "patient_region" in recent_predictions.columns else "region"

if has_region:
    print(f"\n🌎 Regional Performance:")
    regional_analysis = recent_predictions.groupBy(region_col).agg(
        count("*").alias("count"),
        avg("adjusted_prediction").alias("avg_risk"),
        stddev("adjusted_prediction").alias("risk_std"),
        (sum(when(col("high_risk_patient") == True, 1).otherwise(0)) / count("*") * 100).alias("high_risk_pct")
    ).orderBy(region_col).collect()

    print(f"  {'Region':<15} {'Avg Risk':<12} {'Std Dev':<12} {'High-Risk %':<12} {'Count':<10}")
    print(f"  {'-'*70}")
    for row in regional_analysis:
        region_name = getattr(row, region_col)
        print(f"  {region_name:<15} {row.avg_risk:<12.2f} {row.risk_std:<12.2f} {row.high_risk_pct:<12.1f} {row['count']:<10,}")

    # Calculate regional disparity using builtins to avoid PySpark function conflicts
    regional_risks = [row.avg_risk for row in regional_analysis]
    if len(regional_risks) > 1:
        max_risk = builtins.max(regional_risks)
        min_risk = builtins.min(regional_risks)
        disparity = ((max_risk - min_risk) / min_risk) * 100
        
        print(f"\n  Regional Disparity: {disparity:.1f}%")
        if disparity <= 15:
            print(f"  ✅ Excellent regional equity (<15% disparity)")
        elif disparity <= 25:
            print(f"  ✓ Acceptable regional equity (<25% disparity)")
        else:
            print(f"  ⚠️  High regional disparity - investigate for bias")
    else:
        disparity = 0
else:
    print(f"\n🌎 Regional Performance: Region column not available in predictions table")
    regional_risks = []
    disparity = 0

# Gender equity - check if gender column exists
has_gender = "patient_gender" in recent_predictions.columns or "sex" in recent_predictions.columns
gender_col = "patient_gender" if "patient_gender" in recent_predictions.columns else "sex"

if has_gender:
    print(f"\n⚖️  Gender Equity:")
    gender_analysis = recent_predictions.groupBy(gender_col).agg(
        count("*").alias("count"),
        avg("adjusted_prediction").alias("avg_risk"),
        (sum(when(col("high_risk_patient") == True, 1).otherwise(0)) / count("*") * 100).alias("high_risk_pct")
    ).collect()

    for row in gender_analysis:
        gender_name = getattr(row, gender_col)
        print(f"  {gender_name}: Avg Risk={row.avg_risk:.2f}, High-Risk={row.high_risk_pct:.1f}%, N={row['count']:,}")

    gender_risks = {getattr(row, gender_col): row.avg_risk for row in gender_analysis}
    if len(gender_risks) == 2:
        # Use builtins.abs to avoid PySpark function conflict
        gender_disparity = builtins.abs(list(gender_risks.values())[0] - list(gender_risks.values())[1])
        print(f"\n  Gender disparity: {gender_disparity:.2f} risk points")
        if gender_disparity <= 8:
            print(f"  ✅ Excellent gender equity")
        elif gender_disparity <= 15:
            print(f"  ✓ Acceptable gender equity")
        else:
            print(f"  ⚠️  Review for potential gender bias")
else:
    print(f"\n⚖️  Gender Equity: Gender column not available in predictions table")

print("="*80)

## 6. Compliance & Governance

Model governance status, compliance tags, and regulatory readiness.

In [ ]:
print("\n" + "="*80)
print("COMPLIANCE & GOVERNANCE")
print("="*80)

# Model governance status
try:
    champion_info = client.get_model_version_by_alias(full_model_name, "champion")
    
    print(f"\n🏛️  Model Governance Status:")
    print(f"  • Champion Version: {champion_info.version}")
    print(f"  • Status: {champion_info.status}")
    print(f"  • Created: {champion_info.creation_timestamp}")
    print(f"  • Run ID: {champion_info.run_id}")
    
    # Get performance metrics
    run_data = client.get_run(champion_info.run_id)
    metrics = run_data.data.metrics
    
    print(f"\n📊 Champion Model Metrics:")
    print(f"  • R² Score: {metrics.get('r2_score', 0):.4f}")
    print(f"  • Mean Absolute Error: {metrics.get('mean_absolute_error', 0):.2f}")
    print(f"  • High-Risk Accuracy: {metrics.get('high_risk_accuracy', 0):.4f}")
    
    # Healthcare requirements validation
    r2_pass = metrics.get('r2_score', 0) >= 0.70
    mae_pass = metrics.get('mean_absolute_error', 999) <= 15.0
    hr_acc_pass = metrics.get('high_risk_accuracy', 0) >= 0.60
    
    print(f"\n✅ Healthcare Requirements:")
    print(f"  {'✓' if r2_pass else '✗'} R² Score ≥ 0.70: {metrics.get('r2_score', 0):.4f}")
    print(f"  {'✓' if mae_pass else '✗'} MAE ≤ 15.0: {metrics.get('mean_absolute_error', 999):.2f}")
    print(f"  {'✓' if hr_acc_pass else '✗'} High-Risk Accuracy ≥ 0.60: {metrics.get('high_risk_accuracy', 0):.4f}")
    
    # Governance tags
    if champion_info.tags:
        print(f"\n🏷️  Governance Tags:")
        compliance_tags = {k: v for k, v in champion_info.tags.items() 
                          if k in ['healthcare_compliance', 'hipaa_compliant', 'validation_r2', 'model_purpose']}
        for key, value in compliance_tags.items():
            print(f"  • {key}: {value}")
    
    # Description check
    if champion_info.description:
        required_terms = ['Healthcare', 'HIPAA', 'Performance']
        missing_terms = [term for term in required_terms if term not in champion_info.description]
        
        if not missing_terms:
            print(f"\n  ✅ Model description contains all required compliance information")
        else:
            print(f"\n  ⚠️  Model description missing terms: {missing_terms}")
    
    governance_status = "COMPLIANT" if (r2_pass and mae_pass and hr_acc_pass) else "REQUIRES_REVIEW"
    
except Exception as e:
    print(f"\n⚠️  Unable to retrieve champion model: {e}")
    governance_status = "UNKNOWN"

print(f"\n📋 Governance Status: {governance_status}")

print("="*80)

## 7. Recommendations & Action Items

Prioritized recommendations based on validation findings.

In [ ]:
print("\n" + "="*80)
print("RECOMMENDATIONS & ACTION ITEMS")
print("="*80)

recommendations = []

# Business recommendations
if high_risk_pct < 5:
    recommendations.append(("HIGH", "Business", "Increase high-risk identification threshold to ensure adequate intervention candidates"))
elif high_risk_pct > 25:
    recommendations.append(("HIGH", "Business", "Review high-risk threshold to prevent alert fatigue and optimize intervention resources"))

if roi < 50:
    recommendations.append(("HIGH", "Business", "Optimize intervention strategy to improve ROI and business value"))

# Clinical recommendations
if not age_valid:
    recommendations.append(("MEDIUM", "Clinical", "Review age feature engineering - correlation with risk should be monotonic"))

if "yes" in smoker_risks and "no" in smoker_risks and smoker_risks["yes"] <= smoker_risks["no"]:
    recommendations.append(("HIGH", "Clinical", "Smoking feature not properly weighted - recalibrate model"))

# Equity recommendations
if len(regional_risks) > 1 and disparity > 25:
    recommendations.append(("MEDIUM", "Equity", "High regional disparity detected - investigate for potential geographic bias"))

# Technical recommendations
if quality_metrics.null_pct > 1:
    recommendations.append(("MEDIUM", "Technical", "Address data quality issues - null prediction rate above threshold"))

if quality_metrics.outlier_pct > 2:
    recommendations.append(("LOW", "Technical", "Review outlier predictions to ensure model calibration"))

# Positive findings
if not recommendations:
    recommendations.append(("INFO", "Success", "No critical issues identified - continue monitoring and optimization"))

# Sort and display
priority_order = {"HIGH": 1, "MEDIUM": 2, "LOW": 3, "INFO": 4}
recommendations.sort(key=lambda x: priority_order.get(x[0], 99))

print(f"\n📌 Prioritized Action Items:\n")
for priority, category, recommendation in recommendations:
    icon = "🔴" if priority == "HIGH" else "🟡" if priority == "MEDIUM" else "🟢" if priority == "LOW" else "ℹ️"
    print(f"{icon} [{priority:>6}] {category:>10}: {recommendation}")

print("="*80)

## 8. Final Report Summary

Complete validation scorecard and final assessment.

In [ ]:
print("\n" + "="*80)
print("FINAL VALIDATION SCORECARD")
print("="*80)

# Import builtins to avoid conflicts with PySpark functions
import builtins

# Calculate overall scores
validation_categories = {
    "Technical Validation": quality_metrics.null_pct < 1 and quality_metrics.outlier_pct < 2,
    "Business Performance": 5 <= high_risk_pct <= 25 and roi > 50,
    "Clinical Relevance": passed_clinical >= 2,
    "Regional Equity": disparity <= 25 if builtins.len(regional_risks) > 1 else True,
    "Model Governance": governance_status == "COMPLIANT"
}

passed_categories = builtins.sum([1 for result in validation_categories.values() if result])
total_categories = builtins.len(validation_categories)

print(f"\nValidation Results by Category:\n")
for category, result in validation_categories.items():
    status = "✅ PASS" if result else "⚠️  REVIEW"
    print(f"  {status:>12} | {category}")

overall_score = (passed_categories / total_categories) * 100

print(f"\n{'='*80}")
print(f"OVERALL VALIDATION SCORE: {overall_score:.0f}% ({passed_categories}/{total_categories} categories passed)")
print(f"{'='*80}")

# Final assessment
if overall_score >= 90:
    final_status = "EXCELLENT"
    icon = "🟢"
    message = "Model is performing excellently and ready for production use."
elif overall_score >= 70:
    final_status = "GOOD"
    icon = "🟢"
    message = "Model is performing well with minor areas for improvement."
elif overall_score >= 50:
    final_status = "ACCEPTABLE"
    icon = "🟡"
    message = "Model is acceptable but requires attention in several areas."
else:
    final_status = "NEEDS_IMPROVEMENT"
    icon = "🔴"
    message = "Model requires significant improvement before production use."

print(f"\n{icon} FINAL ASSESSMENT: {final_status}")
print(f"\n{message}")

print(f"\n📊 Key Metrics Summary:")
print(f"  • Patients Evaluated: {exec_summary.unique_patients:,}")
print(f"  • High-Risk Identified: {exec_summary.high_risk_count:,}")
print(f"  • Estimated ROI: {roi:.0f}%")
print(f"  • Model Accuracy (R²): {metrics.get('r2_score', 0):.3f}" if 'metrics' in locals() else "")
print(f"  • Data Quality: {100 - quality_metrics.null_pct:.1f}%")

print(f"\n📅 Report Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📋 Report Period: Last {report_period} days")

print("\n" + "="*80)
print("END OF EXECUTIVE VALIDATION REPORT")
print("="*80)

# Return summary - use builtins to avoid PySpark function conflicts
report_summary = {
    "status": final_status,
    "overall_score": overall_score,
    "categories_passed": passed_categories,
    "total_categories": total_categories,
    "high_priority_actions": builtins.len([r for r in recommendations if r[0] == "HIGH"]),
    "roi": builtins.round(roi, 2),
    "patients_evaluated": builtins.int(exec_summary.unique_patients),
    "high_risk_identified": builtins.int(exec_summary.high_risk_count),
    "report_date": datetime.now().isoformat()
}

dbutils.notebook.exit(json.dumps(report_summary))